In [ ]:
import torch
import heapq
import pickle
import time
import json
import numpy as np
import pandas as pd
import os
import cProfile
import pstats
from pathlib import Path
from tqdm.auto import tqdm
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from psutil import cpu_count

SAE_PATH      = Path('out/sae_65k_lambda26_ramp30/sae_final.pt')
TOP_N         = 50
DUMP_DIR      = Path('feature_dumps')
DUMP_DIR.mkdir(exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)
# ---- Load SAE (replace with your own class/loader) ----
from model import SAE
state_dict, config = torch.load(SAE_PATH, map_location=device).values()
sae = SAE(config['input_size'],config['hidden_size']).to(device).to(torch.bfloat16)
# Fix for "_orig_mod" prefix in state dict keys
fixed_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
sae.load_state_dict(fixed_state_dict)

n_features = sae.encode.out_features if hasattr(sae.encode,'out_features') else sae.n_features
print(f'Loaded SAE with {n_features} features')

def count_dead_features(sample_iter, sample_tokens=10_000_000):
    """Returns a boolean tensor of shape (n_features,) where True == dead."""
    fired = torch.zeros(n_features, dtype=torch.bool, device=device)
    seen  = 0
    for toks in tqdm(sample_iter, total=sample_tokens//len(next(iter(sample_iter)))):
        toks = toks.to(device)
        acts = sae.encode(toks) > 0  # bool mask of activations
        fired |= acts.any(dim=0)
        seen  += toks.size(0)
        if seen >= sample_tokens:
            break
    dead_mask = ~fired.cpu()
    print(f"Dead features: {dead_mask.sum().item()} / {n_features} ({dead_mask.float().mean()*100:.2f}%)")
    return dead_mask

# GPU optimizations
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')

# Constants
MODEL_NAME = "allenai/OLMo-2-1124-7B-Instruct"
BATCH_SIZE = 256
LAYER_OFFSET = -1
TOP_N = 50
device = "cuda"
DUMP_DIR = Path("./results")
os.makedirs(DUMP_DIR, exist_ok=True)

def main():
    # Load tokenizer + *half* model config (bf16, compiled)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    cfg = AutoModelForCausalLM.from_pretrained(MODEL_NAME).config
    cfg.num_hidden_layers //= 2  # half-model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        config=cfg,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
        device_map=device
    ).eval()
    model = torch.compile(model, mode="reduce-overhead")

    # Load dataset
    dataset_iter = load_dataset(
        "HuggingFaceFW/fineweb",
        name="sample-10BT",
        split="train",
        streaming=False,
        num_proc=cpu_count(),
    ).shuffle()

    def residual_stream_iter(text_iter, batch=BATCH_SIZE):
        buf = []
        for record in text_iter:
            buf.extend(tokenizer(record["text"]).input_ids)
            while len(buf) >= batch:
                toks = torch.tensor(buf[:batch]).to(device)
                buf = buf[batch:]
                with torch.inference_mode():
                    outs = model(toks.unsqueeze(0), output_hidden_states=True)
                    resid = outs.hidden_states[LAYER_OFFSET].squeeze(0)  # (T, d)
                yield toks, resid  # feed straight to SAE

    def mine_top_tokens_and_dead(data_iter,
                              top_n=TOP_N,
                              target_tokens=100_000):
        """
        • Keeps the TOP-N strongest (activation, token) pairs per feature.
        • Tracks which features ever fire to flag the 'dead' ones.
        • Stops after `target_tokens` have been processed.
        """
        n_features = sae.encode.weight.shape[0]
        
        # Pre-allocate all buckets with empty heaps
        buckets = [[] for _ in range(n_features)]
        fired = torch.zeros(n_features, dtype=torch.bool, device=device)
        seen_toks = 0
        
        # Process batches with lighter progress indicator 
        start_time = time.time()
        batch_count = 0
        
        for toks, resid in data_iter:
            batch_count += 1
            if batch_count % 10 == 0:
                elapsed = time.time() - start_time
                tokens_per_sec = seen_toks / elapsed if elapsed > 0 else 0
                print(f"\rProcessed {seen_toks} tokens ({tokens_per_sec:.1f} tok/s)", end="")
                
            with torch.inference_mode():
                # Compute activations
                acts = sae.encode(resid)
                fired |= (acts > 0).any(dim=0)
                
                # Get top values and indices
                values, idx = acts.topk(1, dim=0)
                
                # Transfer to CPU in one batch  
                values_cpu = values[0].to(torch.float32).detach().cpu().numpy()
                indices_cpu = idx[0].detach().cpu().numpy()
                token_ids = toks[indices_cpu].cpu().numpy()
                
                # Process features in chunks for better performance
                for f in range(n_features):
                    val, tok_id = float(values_cpu[f]), int(token_ids[f])
                    heap = buckets[f]
                    if len(heap) < top_n:
                        heapq.heappush(heap, (val, tok_id))
                    elif val > heap[0][0]:
                        heapq.heapreplace(heap, (val, tok_id))
            
            seen_toks += toks.numel()
            if seen_toks >= target_tokens:
                break
        
        print(f"\nProcessed {seen_toks} tokens in {time.time() - start_time:.2f}s")
        
        # Post-process
        dead_mask = ~fired.cpu().numpy()  # Convert directly to numpy
        
        # Get unique token IDs for batch decoding
        unique_token_ids = set()
        for heap in buckets:
            for _, tok_id in heap:
                unique_token_ids.add(tok_id)
        
        # Convert set to list for batch decoding
        unique_token_list = list(unique_token_ids)
        decoded_tokens = tokenizer.batch_decode([[t] for t in unique_token_list])
        
        # Create mapping of token ID to decoded text
        token_id_to_text = {unique_token_list[i]: decoded_tokens[i] for i in range(len(unique_token_list))}
        
        # Create the final result with native Python types
        decoded = []
        for heap in buckets:
            feature_results = []
            for val, tok_id in sorted(heap, key=lambda x: -x[0]):  # Sort directly here
                feature_results.append((float(val), token_id_to_text[tok_id]))
            decoded.append(feature_results)  # Already sorted
        
        print(f"Dead features: {dead_mask.sum()} / {n_features} "
              f"({dead_mask.sum()/n_features*100:.2f}%)")
        
        return decoded, dead_mask

    data_iter = residual_stream_iter(dataset_iter)
    target_tokens = 50_000_000
    with torch.inference_mode():
        top_buckets, dead_mask = mine_top_tokens_and_dead(
            data_iter,
            top_n=50,
            target_tokens=50_000_000
        )

    # Save results - using faster formats where possible
    # Use pickle for faster serialization
    with open(DUMP_DIR / f"top_tokens_{target_tokens}.pkl", "wb") as f:
        pickle.dump(top_buckets, f)
    
    with open(DUMP_DIR / f"top_tokens_{target_tokens}.json", "w") as f:
        json.dump(top_buckets, f)
    
    np.save(DUMP_DIR / f"dead_features_{target_tokens}.npy", dead_mask)
    
    pd.Series(dead_mask).to_csv(DUMP_DIR / f"dead_features_{target_tokens}.csv", index=False)

# Run with profiling
if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()
    
    main()
    
    profiler.disable()
    
    # Save stats to a file
    stats = pstats.Stats(profiler)
    stats.sort_stats('cumtime')
    stats.dump_stats('profile_results.prof')
    
    print("\n\n--- Profiling Results ---")
    stats.sort_stats('cumtime').print_stats(20)
    
    print("\n\n--- Profiling Results by Function Calls ---")
    stats.sort_stats('calls').print_stats(20)

Device: cuda


Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "/tmp/ipykernel_1253868/3328758826.py", line 210, in <module>
RuntimeError: Cannot install a profile function while another profile function is being installed


Loaded SAE with 65536 features


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]
Some weights of the model checkpoint at allenai/OLMo-2-1124-7B-Instruct were not used when initializing Olmo2ForCausalLM: ['model.layers.16.mlp.down_proj.weight', 'model.layers.16.mlp.gate_proj.weight', 'model.layers.16.mlp.up_proj.weight', 'model.layers.16.post_attention_layernorm.weight', 'model.layers.16.post_feedforward_layernorm.weight', 'model.layers.16.self_attn.k_norm.weight', 'model.layers.16.self_attn.k_proj.weight', 'model.layers.16.self_attn.o_proj.weight', 'model.layers.16.self_attn.q_norm.weight', 'model.layers.16.self_attn.q_proj.weight', 'model.layers.16.self_attn.v_proj.weight', 'model.layers.17.mlp.down_proj.weight', 'model.layers.17.mlp.gate_proj.weight', 'model.layers.17.mlp.up_proj.weight', 'model.layers.17.post_attention_layernorm.weight', 'model.layers.17.post_feedforward_layernorm.weight', 'model.layers.17.self_attn.k_norm.weight', 'model.layers.17.self_attn.k_proj.weight', 'model.layers.17

Processed 49999104 tokens (2243.5 tok/s)
Processed 50000128 tokens in 22286.87s
Dead features: 2336 / 65536 (3.56%)


--- Profiling Results ---
         13146823416 function calls (13117207214 primitive calls) in 22375.153 seconds

   Ordered by: cumulative time
   List reduced from 15830 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    4.105    4.105 22376.231 22376.231 /tmp/ipykernel_1253868/3328758826.py:66(main)
        1 12310.012 12310.012 22294.526 22294.526 /tmp/ipykernel_1253868/3328758826.py:101(mine_top_tokens_and_dead)
   585940 8301.520    0.014 8301.520    0.014 {method 'cpu' of 'torch._C.TensorBase' objects}
   195314   13.252    0.000  870.551    0.004 /tmp/ipykernel_1253868/3328758826.py:89(residual_stream_iter)
12803479658/12803373515  705.880    0.000  705.909    0.000 {built-in method builtins.len}
585939/390626    2.018    0.000  338.463    0.001 /home/henry/.local/lib/python3.10/site-packages/torch

In [ ]:
import asyncio, pickle, pandas as pd, instructor
from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from tqdm.asyncio import tqdm
from dotenv import load_dotenv
import os
load_dotenv()
client = instructor.from_openai(
    AsyncOpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.environ.get("OPENROUTER_API_KEY"))
)

class FeatureLabel(BaseModel):
    label: str = Field(..., description="≤5-word noun phrase or 'uncertain'")
    chain_of_thought: str = Field(..., description="One-sentence rationale")


def make_prompt(tokens: list[tuple]) -> str:
    token_str = ", ".join(str(tok) for _, tok in tokens)

    return (
        "You will receive several tokens that all activate *the same* hidden feature.\n\n"
        f"Tokens: {token_str}\n\n"
        "Return **only** the following JSON object (no additional text):\n"
        "{\n"
        '  "label": "<≤5-word noun phrase, or \'uncertain\', or \'punctuation\'>",\n'
        '  "chain_of_thought": "<one concise sentence explaining why>"\n'
        "}\n\n"
        "• Use \"uncertain\" if no coherent feature emerges.\n"
        "• Use \"punctuation\" if the tokens are mostly punctuation or formatting marks.\n"
        "• Do not output anything except that JSON object."
    )


top_buckets = pickle.load(open("results/top_tokens_50m.pkl", "rb"))
sem = asyncio.Semaphore(20)

async def label_one(idx, tokens):
    prompt = make_prompt(tokens)
    async with sem:
        fl = await client.chat.completions.create(
            model="google/gemini-2.0-flash-001",
            response_model=FeatureLabel,
            temperature=0.6,
            messages=[{"role":"user","content":prompt}],
        )
    return {
        "feature_id": idx,
        "label": fl.label,
        "chain_of_thought": fl.chain_of_thought,
        "tokens": tokens,
    }

async def main():
    tasks = [asyncio.create_task(label_one(i, t)) for i, t in enumerate(top_buckets)]
    return await tqdm.gather(*tasks)

rows = await main()

df = pd.DataFrame(rows).sort_values("feature_id")
df.to_csv("results/feature_labels.csv", index=False)
df.head()


  7%|▋         | 4296/65536 [02:31<36:05, 28.28it/s]  